# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst): 
    return foldr(lambda x, z: lambda y: z(f(y, x)), lambda j: j, lst)(x0)

In [6]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', ['a', 'b', 'c', 'd'])

'abcd'

In [7]:
def foldr2(f, x0, lst): 
    return foldl(lambda z, x: lambda y: z(f(x, y)), lambda j: j, lst)(x0)

In [8]:
foldr2(lambda x, y: '{}{}'.format(x, y), '', ['a', 'b', 'c', 'd'])

'abcd'

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [9]:
def check_inv(a, b):
    hash_b = 0
    hash_sub_a = 0
    for s in b:
        hash_b += hash(s)
    
    if (len(a) == len(b)):          #  additional check for equel len (this case skip cycle)
        for s in b:                     
            hash_sub_a += hash(s)
        if hash_sub_a == hash_b:
            return True
    
    for i in range(len(a)-len(b)):  #  check for different len
        hash_sub_a = 0
        for j in range(i, i + len(b)):
            hash_sub_a += hash(a[j])
        if hash_sub_a == hash_b:
            return True
    return False
    

check_inv('were', 'erwe') == True

True

In [10]:
check_inv('abcrotm', 'tro')

True

In [19]:
check_inv('fafd', 'fafdgsdfg')

False

In [21]:
check_inv('eeewrre', 'were')

True

In [13]:
check_inv('were', 'were') == True

True

Сложность Θ(n*m) , n = len(a), m = len(b)
>Я понимаю, что в теории этот алгоритм может когда-то дать сбой, если попадётся комбинация символов дающая в сумме равный хэш, но на практике это почти невозможно.

# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [14]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __iter__(self):
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right
        if not self.left and not self.right:
            yield self.value

    def __str__(self):
        return f'Tree Value = {self.value}'

    def __repr__(self):
        return f'Current: {self}\nLeft: [{self.left}]\nRight: [{self.right}]'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

list(tree) == [3, 4, 2]

True

In [15]:
print(tree)

Tree Value = 0


In [16]:
tree

Current: Tree Value = 0
Left: [Tree Value = 1]
Right: [Tree Value = 2]

# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [17]:
import re


def prioritize_operators(stack, x): # compare operators
    order = {'+': 1, '-': 1, '*': 2, '/': 2}
    try:
        return order[x] <= order[stack[-1]]
    except KeyError:
        return False


def order_operations(inputs):
    outputs = [] # result
    stack = [] # for operations in brackets
    for x in inputs:
        if x.isdigit():
            outputs.append(int(x))
        elif x == '(':
            stack.append(x)
        elif x == ')':
            while stack and stack[-1] != '(':
                outputs.append(stack.pop())
            stack.pop()
        else:
            while stack and prioritize_operators(stack, x):
                outputs.append(stack.pop())
            stack.append(x)
    while stack:
        outputs.append(stack.pop())
    return outputs


def calc(expr):
    elements = re.findall(r'[-+/*()]|\d+(?:\.\d+)?', expr) # parse elements to list
    ordered_el = order_operations(elements) # order elements using brackets

    stack = []
    for x in ordered_el:
        if type(x) is int:
            stack.append(x)
        else:
            el2, el1 = stack.pop(), stack.pop()

            if x == '+':
                stack.append(el1 + el2)
            elif x == '-':
                stack.append(el1 - el2)
            elif x == '*':
                stack.append(el1 * el2)
            elif x == '/':
                stack.append(el1 // el2)

    return stack.pop()

calc('2 * (15 - 3 * 4) - 2') == 4

True

In [18]:
calc('7*7+(13)') == 62

True